In [1]:
import pyodbc
import dotenv
import os
import sys
import pandas as pd

os.chdir('../')
dotenv.load_dotenv()
import sql
from places import get_places
from store import Store

In [2]:
from defaults import *
stores = []
zipcodes = zipcodes[:1]
store_types = store_types[:1]
for zipcode in zipcodes:
    for store_type in store_types:
        query = f'{store_type}+in+{zipcode}'
        for place in get_places(query):
            stores.append(Store(place))

In [3]:
sql.set_up()
#sql.store_to_db(stores[0])
stores[0].is_triggering= True
sql.store_flag(stores[0])

# In notebook testing to produce function 

In [4]:
from sqlalchemy import text
sql.set_up()
place = stores[0].place
query = f'''MERGE INTO stores
USING ( VALUES(\'{place['place_id']}\')) AS source (place_id)
ON stores.place_id = source.place_id
WHEN MATCHED THEN
UPDATE SET last_updated = GETDATE() AT TIME ZONE 'UTC'
WHEN NOT MATCHED THEN
INSERT (place_id)
VALUES (\'{place['place_id']}\');'''.replace('\n',' ')
sql.conn.execute(text(query))

In [5]:
sql.conn.commit()

In [6]:
dir(stores[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'image_path',
 'image_text',
 'is_triggering',
 'photos',
 'place',
 'reviews',
 'save',
 'website']

In [19]:

# Need Update with additional flags
# Right now working with is_triggering to set flag_text
def store_update(s: Store):
    # Only update when one of the flags are True
    if s.is_triggering == True:
        query = f'''UPDATE stores 
        SET last_flagged = GETDATE() AT TIME ZONE 'UTC',
        flag_text = {int(s.is_triggering) if s.is_triggering else 0}
        WHERE place_id=\'{s.place['place_id']}\''''
        conn.execute(text(query))
        conn.commit()

In [16]:
s=stores[0]
s.is_triggering = False
query = f'''UPDATE stores 
SET last_flagged = GETDATE() AT TIME ZONE 'UTC',
flag_text = {int(s.is_triggering) if s.is_triggering else 0}
WHERE place_id=\'{s.place['place_id']}\''''
sql.conn.execute(sql.text(query))
sql.conn.commit()

In [17]:
None == True

False

In [5]:
pd.read_sql('SELECT * FROM [dbo].[raw_permit]', sql.conn)